In [1]:
import optuna
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("D:\My Downloads\winequality-red.csv-main\winequality-red.csv-main\winequality-red.csv")

In [3]:
df.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


In [4]:
# There are no null values, no need to do null imputation. Lets split the data

x = df.drop(columns=['quality'])

In [5]:
y = df.quality

In [13]:
y

0       5
1       5
2       5
3       6
4       5
       ..
1594    5
1595    6
1596    6
1597    5
1598    6
Name: quality, Length: 1599, dtype: int64

In [6]:
train_x, test_x, train_y, test_y = train_test_split(x,y, test_size = 0.25, random_state = 30)

In [7]:
train_x

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
1178,5.6,0.915,0.00,2.1,0.041,17.0,78.0,0.99346,3.68,0.73,11.4
1189,8.8,0.955,0.05,1.8,0.075,5.0,19.0,0.99616,3.30,0.44,9.6
161,7.6,0.680,0.02,1.3,0.072,9.0,20.0,0.99650,3.17,1.08,9.2
109,8.1,0.785,0.52,2.0,0.122,37.0,153.0,0.99690,3.21,0.69,9.3
961,7.1,0.560,0.14,1.6,0.078,7.0,18.0,0.99592,3.27,0.62,9.3
...,...,...,...,...,...,...,...,...,...,...,...
919,8.4,0.620,0.12,1.8,0.072,38.0,46.0,0.99504,3.38,0.89,11.8
1164,9.0,0.785,0.24,1.7,0.078,10.0,21.0,0.99692,3.29,0.67,10.0
500,7.8,0.520,0.25,1.9,0.081,14.0,38.0,0.99840,3.43,0.65,9.0
941,9.9,0.270,0.49,5.0,0.082,9.0,17.0,0.99484,3.19,0.52,12.5


In [8]:
np.asarray(train_x)

array([[ 5.6  ,  0.915,  0.   , ...,  3.68 ,  0.73 , 11.4  ],
       [ 8.8  ,  0.955,  0.05 , ...,  3.3  ,  0.44 ,  9.6  ],
       [ 7.6  ,  0.68 ,  0.02 , ...,  3.17 ,  1.08 ,  9.2  ],
       ...,
       [ 7.8  ,  0.52 ,  0.25 , ...,  3.43 ,  0.65 ,  9.   ],
       [ 9.9  ,  0.27 ,  0.49 , ...,  3.19 ,  0.52 , 12.5  ],
       [ 6.6  ,  0.84 ,  0.03 , ...,  3.52 ,  0.56 , 12.3  ]])

In [9]:
xgb_classification = xgb.XGBClassifier(**param)
xgb_classification.fit(train_x , train_y , eval_set = [(test_x, test_y)])

NameError: name 'param' is not defined

In [16]:
def objective_classification(trial) :
    train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.20)
    
    param = {
        'tree_method' : 'gpu_hist',
        'verbosity' : 3 ,
        'objective' : "binary:logistics",
        'booster' : trial.suggest_categorical('booster' , ['dart' ,'gbtree' , 'gblinear']),
        'lambda' : trial.suggest_float('lambda' , 1e-4, 1),
        'alpha' : trial.suggest_float('alpha' , 1e-4, 1),
        'subsample' : trial.suggest_float('subsample' , .1, .5),
        'colsmaple_bytree' : trial.suggest_float('colsample_bytree' , .1 , .5)
    }
    
    if param['booster' ] in ['gbtree' , 'dart']:
        param['gamma'] : trial.suggest_float('gamma' , 1e-3, 4)
        param['eta'] : trial.suggest_float('eta' , .001,5)
    
    xgb_classification = xgb.XGBClassifier(**param)
    xgb_classification.fit(train_x , train_y , eval_set = [(test_x, test_y)])
    pred = xgb_classification.predict(test_x)
    accuracy = xgb_classification.score(test_x, test_y)
    
    return accuracy

In [17]:
xgb_classification_optuna = optuna.create_study(direction='minimize')
xgb_classification_optuna.optimize(objective_classification,  n_trials = 10)

[I 2022-06-15 23:37:14,945] A new study created in memory with name: no-name-a4e0cea9-8b40-4193-9e55-f25f70d7e981
[W 2022-06-15 23:37:14,950] Trial 0 failed because of the following error: ValueError('Invalid classes inferred from unique values of `y`.  Expected: [0 1 2 3 4 5], got [3 4 5 6 7 8]')
Traceback (most recent call last):
  File "C:\Users\anand\Anaconda3\lib\site-packages\optuna\study\_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-16-3fd512423b88>", line 20, in objective_classification
    xgb_classification.fit(train_x , train_y , eval_set = [(test_x, test_y)])
  File "C:\Users\anand\Anaconda3\lib\site-packages\xgboost\core.py", line 532, in inner_f
    return f(**kwargs)
  File "C:\Users\anand\Anaconda3\lib\site-packages\xgboost\sklearn.py", line 1358, in fit
    f"Invalid classes inferred from unique values of `y`.  "
ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0 1 2 3 4 5], got [3 4 5 6 7 8]


ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0 1 2 3 4 5], got [3 4 5 6 7 8]

In [ ]:
xgb_classification_optuna.best_trial

In [ ]:
param = xgb_classification_optuna.best_trial.params

In [ ]:
xgb_final_clas = xgb.XGBClassifier(**param)
xgb_final_clas.fit(train_x, train_y)

In [ ]:
xgb_final_clas.score(test_x, test_y)